In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
datasets = {}
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        datasets[filename] = os.path.join(dirname, filename)
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv(datasets["train.csv"])
test = pd.read_csv(datasets["test.csv"])
train.head()


In [ ]:
train.shape

In [ ]:
train.isnull().any()
test.isnull().any()

In [ ]:
mask1 = train["Embarked"].notna()
train = train[mask1]
train.head()

test = test[test["Embarked"].notna()]

In [ ]:
train["Age"].fillna(train["Age"].mean(), inplace=True)
train.isnull().any()

test["Age"].fillna(test["Age"].mean(), inplace=True)
test.isnull().any()

In [ ]:
X_train = train[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
y_train = train[["Survived"]]
X_test = test[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]

In [ ]:
X_train = pd.get_dummies(X_train, columns=["Sex", "Embarked"])
X_test = pd.get_dummies(X_test, columns=["Sex", "Embarked"])

In [ ]:
X_test.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import sklearn.naive_bayes 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder

In [ ]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_train, y_train, random_state=910)

In [ ]:
Log_R = LogisticRegression()

In [ ]:
Log_R.fit(X_train_full, y_train)

In [ ]:
Log_R.score(X_valid_full, y_valid)

In [ ]:
a.head()

In [ ]:
X_full = pd.read_csv(datasets["train.csv"])
X_test_full = pd.read_csv(datasets["test.csv"])
X = X_full.copy()
y = X.Survived
X.drop(['Survived'], axis=1, inplace=True)

In [ ]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [ ]:
# Select categorical columns with no more than 15 unique values
categorical_cols = [col for col in X_train_full.columns if X_train_full[col].nunique() <= 15 and X_train_full[col].dtypes == 'object']

# Select numeric values
numerical_cols = [col for col in X_train_full.columns if X_train_full[col].dtype in ['int', 'float']]

# Keep selected columns
my_columns = categorical_cols + numerical_cols
X_train = X_train_full[my_columns].copy()
X_valid = X_valid_full[my_columns].copy()
X_test = X_test_full[my_columns].copy()

In [ ]:
#Preprocessing numerical values
Numerical_transformer = SimpleImputer(strategy="mean")

#preprocessing categorical values
Categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#packs the preprocessor together
preprocessor = ColumnTransformer(transformers=[
    ('num', Numerical_transformer, numerical_cols),
    ('cat', Categorical_transformer, categorical_cols)
])

In [ ]:
model = LogisticRegression()

my_pipeline = Pipeline(steps = [('preprocess', preprocessor),
                               ('model', model)])

my_pipeline.fit(X_train, y_train)
preds = my_pipeline.predict(X_valid)
#print('MAE:', mean_absolute_error(y_valid.to_numpy().reshape(134,), preds))
print('Score:', my_pipeline.score(X_valid, y_valid.to_numpy().ravel()))

In [ ]:
kfold = KFold(shuffle = True, random_state = 0)

scores = cross_validate(my_pipeline, X_train, y_train, scoring = 'neg_mean_squared_error', cv=kfold)

print('Average MSE score', (scores['test_score']*-1).mean())

In [ ]:
X_train.head()

**GridSearch CV**

In [ ]:
param_grid = {
    'model__max_iter' : [500, 1000, 1500, 2000],
    'model__solver': ['lbfgs', 'newton-cg', 'sag', 'saga']}

kfold = KFold(shuffle=True, random_state=0)
grid_search = GridSearchCV(my_pipeline, param_grid, scoring = 'neg_mean_squared_error', cv=kfold)
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_score_

In [ ]:
XGmodel = XGBClassifier(verbosity = 0, random_state = 0)

my_pipeline_XG = Pipeline(steps = [('preprocess', preprocessor),
                               ('model', XGmodel)])

my_pipeline_XG.fit(X_train, y_train)
preds = my_pipeline_XG.predict(X_valid)
#print('MAE:', mean_absolute_error(y_valid.to_numpy().reshape(134,), preds))
print('Score:', my_pipeline_XG.score(X_valid, y_valid.to_numpy().ravel()))

In [ ]:
kfold = KFold(shuffle = True, random_state = 0)

scores = cross_validate(my_pipeline_XG, X_train, y_train, scoring = 'neg_mean_squared_error', cv=kfold)

print('Average MSE score', (scores['test_score']*-1).mean())

In [ ]:
param_grid = {'model__n_estimators': [10, 50, 100, 200, 400, 600],
              'model__max_depth': [2, 3, 5, 7, 10],
              'model__min_child_weight': [0.0001, 0.001, 0.01],
              'model__learning_rate': [0.01, 0.1, 0.5, 1]}

kfold = KFold(shuffle=True, random_state=0)
grid_search = GridSearchCV(my_pipeline_XG, param_grid, scoring = 'neg_mean_squared_error', cv=kfold)
grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_score_)
print(grid_search.best_params_)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
model = KNeighborsClassifier()

my_pipelineKNN = Pipeline(steps = [('preprocess', preprocessor),
                               ('model', model)])

my_pipelineKNN.fit(X_train, y_train)
preds = my_pipelineKNN.predict(X_valid)
print('MSE:', mean_squared_error(y_valid, preds))
print('Score:', my_pipelineKNN.score(X_valid, y_valid.to_numpy().ravel()))

In [ ]:
model = SVC(gamma='auto')

my_pipelineSVC = Pipeline(steps = [('preprocess', preprocessor),
                               ('model', model)])

my_pipelineSVC.fit(X_train, y_train)
preds = my_pipelineSVC.predict(X_valid)
print('MSE:', mean_squared_error(y_valid, preds))
print('Score:', my_pipelineSVC.score(X_valid, y_valid))

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

my_pipelineNB = Pipeline(steps = [('preprocess', preprocessor),
                               ('model', model)])

my_pipelineNB.fit(X_train, y_train)
preds = my_pipelineNB.predict(X_valid)
print('MSE:', mean_squared_error(y_valid, preds))
print('Score:', my_pipelineNB.score(X_valid, y_valid))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis()

my_pipelineLDA = Pipeline(steps = [('preprocess', preprocessor),
                               ('model', model)])

my_pipelineLDA.fit(X_train, y_train)
preds = my_pipelineLDA.predict(X_valid)
print('MSE:', mean_squared_error(y_valid, preds))
print('Score:', my_pipelineLDA.score(X_valid, y_valid))

In [ ]:
param_grid = {'model__solver': ['svd', 'lsqr', 'eigen'],
              'model__shrinkage': [None, 'auto']}

kfold = KFold(shuffle=True, random_state=0)
grid_searchLDA = GridSearchCV(my_pipelineLDA, param_grid, scoring = 'neg_mean_squared_error', cv=kfold)
grid_searchLDA.fit(X_train, y_train)

In [ ]:
grid_searchLDA.best_params_

In [ ]:
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier()

my_pipelineBC = Pipeline(steps = [('preprocess', preprocessor),
                               ('model', model)])

my_pipelineBC.fit(X_train, y_train)
preds = my_pipelineBC.predict(X_valid)
print('MSE:', mean_squared_error(y_valid, preds))
print('Score:', my_pipelineBC.score(X_valid, y_valid))

In [ ]:
param_grid = {
             'model__base_estimator' : [None, SVC, KNeighborsClassifier, LinearDiscriminantAnalysis, XGBClassifier, LogisticRegression, GaussianNB],
            'model__n_estimators': [10,20,30,50,100],
        'model__max_samples' : [0.05, 0.1, 0.2, 0.5],
    'model__max_features': [0.3, 0.5, 0.7, 0.9, 1.0],
    'model__bootstrap': [True, False],
    'model__bootstrap_features': [True, False],
    'model__warm_start': [True, False],
             }

kfold = KFold(shuffle=True, random_state=0)
grid_searchBC = GridSearchCV(my_pipelineBC, param_grid, scoring = 'neg_mean_squared_error', cv=kfold)
grid_searchBC.fit(X_train, y_train)
print("best param: ", grid_searchBC.best_params_)
print("Best score: ", grid_searchBC.best_score_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)

my_pipelineRFC = Pipeline(steps = [('preprocess', preprocessor),
                               ('model', model)])

my_pipelineRFC.fit(X_train, y_train)
preds = my_pipelineRFC.predict(X_valid)
print('MSE:', mean_squared_error(y_valid, preds))
print('Score:', my_pipelineRFC.score(X_valid, y_valid))

In [ ]:
param_grid = {
             'model__n_estimators': [10,20,50],
        'model__criterion' : ['gini', 'entropy', 'log_loss'],
    'model__max_depth': [None, 3, 5, 7, 10],
    'model__max_features': ['sqrt', 'log2', None],
    'model__max_leaf_nodes': [0, 1, 5, 10],
    'model__min_impurity_decrease' : [0, 0.001, 0.01],
    'model__ccp_alpha': [0,  0.001, 0.1],
             }

kfold = KFold(shuffle=True, random_state=0)
grid_searchRFC = GridSearchCV(my_pipelineRFC, param_grid, scoring = 'neg_mean_squared_error', cv=kfold)
grid_searchRFC.fit(X_train, y_train)
print("best param: ", grid_searchRFC.best_params_)
print("Best score: ", grid_searchRFC.best_score_)

In [ ]:
#predict test values

preds = my_pipelineRFC.predict(X_test)
preds

In [ ]:
submission = pd.DataFrame({'PassengerId': X_test["PassengerId"], 'Survived': preds})
submission.head()

In [ ]:
filename = 'Titanic Predictions 1.csv'
submission.to_csv(filename, index=False)
print("Saved file :", filename)